In [1]:
import pandas as pd
import requests
import tensorflow as tf
from sklearn.utils import resample
from tensorflow import keras
from bs4 import BeautifulSoup
import re
from sklearn.model_selection import train_test_split
import random
from random import sample
import json
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from spacy.lang.de import German
from utils import *

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jannis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jannis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
nlp=German()
doc=nlp(text)
nlp.add_pipe('sentencizer')
sents=list(doc.sents)
print(sents)

ValueError: [E088] Text of length 1324112 exceeds maximum of 1000000. The parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.

In [16]:
from datasets import load_dataset
dataset = load_dataset("mlsum", "de")
df_json=pd.read_json(dataset["train"])
claim=" "
i=0
while i<10000:
    claim=claim+dataset["train"]["summary"][randint(0,220000)]
    i=+1
array=dataset["train"]["summary"][:10000]
s=" ".join(array)


ValueError: Invalid file path or buffer object type: <class 'datasets.arrow_dataset.Dataset'>

Found cached dataset mlsum (/Users/jannis/.cache/huggingface/datasets/mlsum/de/1.0.0/033c69bbbf1eb198d444f668be75f297cb86251c0671a3d063d1c53c2f231076)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
articles=['Wiesentalbrücke','Mammut (Gattung)','Riemannsche Vermutung','Reichstag zu Augsburg','Deutsch-Französischer Krieg','Kantonsspital Winterthur','Femizid','Nicht-zufällige Segregation von Chromosomen','Beryllium','Massenaussterben','Covid-19','Pandemie','Sex','Homosexualität','OG Keemo','Conchita Wurst','Hochschule für Medien, Kommunikation und Wirtschaft','Europäische Union','Biographie','Donald Trump','Angstzustände','Doktortitel','BAHN-BKK']

In [ ]:
articles=["Maschinelles Lernen", "Medizin","Wissenschaft","Krankheit","Prävention","Diagnose","Politik","COVID-19","COVID-19-Pandemie","Epidemie","Mykose","Sexuell übertragbare Erkrankung","Infektionskrankheit","Bundestag","Bundesrat","Zeitung","Rundfunk","Verlag","Politisches System der Bundesrepublik Deutschland","Politisches System","Massenmedien","Medienwissenschaft","Publikation"]


In [4]:
# Loop over articles

text=''
for name in articles:
    print(name)
    raw=fetch_rawtext_from_wiki(name)
    text=text + raw
df=preprocess_classify_wiki_text(text)



Wiesentalbrücke
Mammut (Gattung)
Riemannsche Vermutung
Reichstag zu Augsburg
Deutsch-Französischer Krieg
Kantonsspital Winterthur
Femizid
Nicht-zufällige Segregation von Chromosomen
Beryllium
Massenaussterben
Covid-19
Pandemie
Sex
Homosexualität
OG Keemo
Conchita Wurst
Hochschule für Medien, Kommunikation und Wirtschaft
Europäische Union
Biographie
Donald Trump
Angstzustände
Doktortitel
BAHN-BKK


In [15]:
train_df.head()

,text,target
1517,"gehören personen ab 60 jahren, immungeschwächt...",False
106,vielfalt gebiete teilgebiete sowie zunahme wis...,True
1449,"weltweite coronawelle, verursacht hochgradig ü...",True
3945,bundesverfassungsgericht parteiverbot verwirku...,False
2485,candida albicans candida-arten anamorphe (impe...,False


In [13]:
#Pipeline, if "claims" is free text of all labeled claims 
split_claims=split_text(claims)
df_claim =pd.DataFrame()
df_claim["text"]= split_claims
df_claim=df_claim.assign(target=True)
df=df.loc[df["target"]==False]
df=pd.concat([df,df_claim])

NameError: name 'claims' is not defined

In [10]:
## Re-Balanced Class Sets
df_majority = df[df.target==0]
df_minority = df[df.target==1]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=2451,     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df = pd.concat([df_majority_downsampled, df_minority])

In [5]:
#Further Pipeline to preprocess model
df["text"]=df.text.map(remove_stopwords)
counter = counter_word(df.text)
num_unique_words=len(counter)
train_df,test_df = split_train_test(df)
print(num_unique_words)
tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(train_df["text"]) 
train_sequences = tokenizer.texts_to_sequences(train_df['text'])
val_sequences = tokenizer.texts_to_sequences(test_df['text'])
train_padded = pad_sequences(train_sequences, maxlen=32, padding="post", truncating="post")
val_padded = pad_sequences(val_sequences, maxlen=32, padding="post", truncating="post")
word_index = tokenizer.word_index
reverse_word_index = dict([(idx, word) for (word, idx) in word_index.items()])


36603


In [44]:
df.describe()

,text,target
count,4902,4902
unique,4841,2
top,B,False
freq,17,2451


In [31]:
df['target'].value_counts().describe()
df.describe(include='all')a

TypeError: NDFrame.describe() got an unexpected keyword argument 'object'

In [6]:
# Create LSTM model
from tensorflow.keras import layers, callbacks
from keras.callbacks import EarlyStopping
# Embedding: https://www.tensorflow.org/tutorials/text/word_embeddings
# Turns positive integers (indexes) into dense vectors of fixed size. (other approach could be one-hot-encoding)

# Word embeddings give us a way to use an efficient, dense representation in which similar words have 
# a similar encoding. Importantly, you do not have to specify this encoding by hand. An embedding is a 
# dense vector of floating point values (the length of the vector is a parameter you specify).

es = EarlyStopping('val_loss',mode='min',verbose=1,patience =2)

model = keras.models.Sequential()
model.add(layers.Embedding(num_unique_words, 32, input_length=32))

# The layer will take as input an integer matrix of size (batch, input_length),
# and the largest integer (i.e. word index) in the input should be no larger than num_words (vocabulary size).
# Now model.output_shape is (None, input_length, 32), where `None` is the batch dimension.


model.add(layers.LSTM(64, dropout=0.1))

model.add(layers.Dense(1, activation="sigmoid"))
#TODO try out others than sigmoid
#min 2nd layer or more

model.summary()
loss = keras.losses.BinaryCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(learning_rate=0.001)
metrics = ["accuracy"]

model.compile(loss=loss, optimizer=optim, metrics=metrics)
model.fit(train_padded, train_df["target"], epochs=20, validation_data=(val_padded, test_df["target"]), verbose=2, callbacks=[es])

2023-01-06 13:36:25.002101: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 32)            1171296   
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 1,196,193
Trainable params: 1,196,193
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
246/246 - 9s - loss: 0.5734 - accuracy: 0.7225 - val_loss: 0.5625 - val_accuracy: 0.7202 - 9s/epoch - 38ms/step
Epoch 2/20
246/246 - 6s - loss: 0.3723 - accuracy: 0.8438 - val_loss: 0.6197 - val_accuracy: 0.7294 - 6s/epoch - 26ms/step
Epoch 3/20
246/246 - 6s - loss: 0.1350 - accuracy: 0.9557 - val_

In [47]:
fang_text=fetch_from_fangcovid_local(5,10)
#fang_pre=preprocess_text(fang_text)
df_fang=pd.DataFrame()
df_fang["sentence"]=split_text(fang_text)
df_fang["tokenized"]=tokenizer.texts_to_sequences(df_fang["sentence"])
fang_padded = pad_sequences(df_fang["tokenized"], maxlen=32, padding="post", truncating="post")
df_fang["predictions"]=model.predict(fang_padded)

3/3 [==============================] - 1s 5ms/step


In [48]:
# df_fang["predictions"]=model.predict(fang_padded)

print(df_fang.sort_values("predictions"))
os.chdir("/Users/jannis/Desktop/fang-covid-main")
df_fang=df_fang[['predictions','sentence']]
df_fang.to_csv("results7.csv" )

                                             sentence  \
65   Darunter das Wellenbad am Spreewaldplatz in K...   
4   \n#Gold: streetTRACKS Gold Shares ETF (GLD) $G...   
76    Die Maximalwerte liegen zwischen 26 und 30 Grad   
62  \n„Wir verstehen, dass es da Unmut gibt und da...   
68   Im vergangenen Jahr hatte man so die Freibads...   
..                                                ...   
36   Die Risiken sind noch dramatischer geworden: ...   
42  \nIst 2019-nCoV eine Biowaffe?\nImmer noch wol...   
41   Diese Einschätzung wird daran festgemacht, da...   
27   Demnach wird alleine die Zahl der Infizierten...   
72   Wie im Sommer können nur Zeittickets erworben...   

                                            tokenized  predictions  
65                            [370, 4380, 578, 16999]     0.004970  
4             [2399, 2399, 13954, 35, 577, 10131, 13]     0.005231  
76                   [1459, 411, 1117, 912, 551, 459]     0.005295  
62  [4642, 1797, 6680, 1400, 127, 912, 

In [1]:
os.getcwd()

'/Users/jannis/Desktop/TryVSCode/Master_Thesis'

In [5]:
label_count={}
claims=" "
os.chdir("/Users/jannis/Desktop/x-fact-main/data/x-fact")
with open("train.all.tsv", 'r') as fp:
    for line in fp:
        arr = line.strip().split('\t')
        lang = arr[1].lower()
        site = arr[2].lower()
        domain = (lang, site)
        

        if domain not in label_count:
                label_count[domain] = {}
        if arr[0] == 'de':
            claims = claims + arr[3]
            print(arr[3])
            

        label = arr[-1].lower()

        if label not in label_count[domain]:
            label_count[domain][label] = 0
        label_count[domain][label] +=1



# new_map = {}
# for key in label_count.keys():
#     new_map[key] = {}
#     counts = label_count[key]

#     total = 0
#     for k, v in counts.items():
#         total += v

#     for k,v in counts.items():
#         new_map[key][k] = float(v)/total


# print(new_map)

# total = 0
# count = 0
# for key in new_map.keys():
#     counts = new_map[key]
#     take = True
#     for k, v in counts.items():
#         if v > 0.7:
#             take = False

#     if take:
#         print(key)
#         count +=1
#         for k, v in label_count[key].items():
#             total += v

# print(total)
# print(count)


Rhein-Neckar-Bahn: Asylbewerber fahren umsonst
Das Coronavirus enthält HIV-Anteile, wurde also im Labor erschaffen
05.05.2020 — Ein Kettenbrief warnt: Kriminelle würden an Haustüren angeblich Masken mit Betäubungsmittel verteilen, um Leute auszurauben. Wir prüfen ...
2019/12/09 — Jedes Jahr kehren Tausende Guineer, die sich Richtung Europa aufgemacht haben, in ihr Land zurück. Viele nehmen an Rückkehrer-Programmen teil und bekommen Unterstützung bei ihrem Neuanfang. Doch die Ankunft in ...
0 Posts - See Instagram photos and videos from 'uiguren' hashtag. ... Top posts. Photo by News von ZDFheute on September 08, 2020. Image may contain: one.
"Michael Schumacher ist tot", lautet die Meldung, die tausendfach gepostet wird.
Apr 11, 2018 â€” Faktencheck. Hessen â€“ Asylbewerber erhalten in AusnahmefÃ¤llen FÃ¼hrerschein mit abgelaufenen Dokumenten. Anerkannte AsylbewerberÂ ...
Der Anteil der Muslime mit türkischem Migrationshintergrund ist von 67,5 Prozent (2011) auf 50,6 Prozent (2015) ges

In [6]:
len(claims)

119818

In [89]:
text=fetch_from_fangcovid(5)
print(len(text))

17574


In [65]:
df=pd.read_csv("https://raw.githubusercontent.com/justusmattern/fang-covid/main/articles/20000.json")


ParserError: Error tokenizing data. C error: Expected 2 fields in line 7, saw 46


Papst Franziskus unterstützt weiterhin die globale Corona-Panikmache. Täglich prasseln sich an Dramatik überbietende „Schreckensmeldungen“ auf die Bevölkerung nieder, die bei näherem Hinsehen meist die Verhältnismäßigkeit vermissen lassen.
Papst Franziskus ließ nun verkünden, daß es auch zu Weihnachten keine Papstmessen geben werde. Obwohl die Corona-Medien die Nachricht bereitwillig in ihr tägliches Corona-Panikpensum aufnehmen, ist sie keine Sensation.
Eine Sensation wäre es gewesen, wenn der Papst angekündigt hätte, endlich wieder die Zelebration öffentlicher Gottesdienste aufzunehmen. Unterdessen erlebt die Kirche im Verhältnis die größte Fahnenflucht ihrer Geschichte.
Das vatikanische Staatssekretariat teilte gestern in einer Note allen beim Heiligen Stuhl akkreditierten Botschaftern mit, daß es zu Weihnachten, dem Geburtsfest Jesu Christi, keine öffentlichen päpstlichen Meßfeiern und protokollarischen Zeremonien geben wird.
Auf den ersten Blick klingt die die Ankündigung des vati

In [59]:
df=fetch_from_fangcovid(0)

ParserError: Error tokenizing data. C error: Expected 2 fields in line 6, saw 3


In [ ]:
def tokenize_text(text):
    nlp = spacy.load("de_core_news_md")
    doc = nlp(sentences[0])
    print(doc.text)
    for token in doc:
        print(token.text, token.pos_, token.dep_)